In [ ]:
!pip3 install mediapipe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 49.9 MB/s eta 0:00:00


In [ ]:
import cv2
import mediapipe as mp
import pandas as pd

In [ ]:
mp_pose = mp.solutions.pose
mp.drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, min_detection_confidence=0.5)

In [ ]:
# Video Output
import time
import cv2
import mediapipe as mp
import numpy as np
from google.colab import files  # For downloading files in Colab

# Initialize MediaPipe Pose
mpDraw = mp.solutions.drawing_utils
mpPose = mp.solutions.pose
pose = mpPose.Pose()

# Input video parameters
width = 900
height = 900

# Upload video file to Colab if needed
# This is commented out because you're already accessing the file from Drive
# uploaded = files.upload()

# You are loading the file from Drive, so no need to iterate through uploaded files
# for filename in uploaded.keys():
#     input_file = '/content/drive/MyDrive/PoseVideo/P1.mp4'  # Get the uploaded video file name

# Directly set the input file path
input_file = '/content/drive/MyDrive/150.mp4'

# Open the video file
cap = cv2.VideoCapture(input_file)

# Prepare for output video
output_file = 'mediapipe_enacted_output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_file, fourcc, 30.0, (width, height))

pTime = 0
while True:
    success, img = cap.read()
    if not success:
        break

    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = pose.process(imgRGB)

    if results.pose_landmarks:
        # Draw landmarks on the original image
        mpDraw.draw_landmarks(img, results.pose_landmarks, mpPose.POSE_CONNECTIONS)

        # Get landmark positions to define the bounding box
        h, w, c = img.shape
        x_min, y_min = w, h
        x_max, y_max = 0, 0

        for id, lm in enumerate(results.pose_landmarks.landmark):
            cx, cy = int(lm.x * w), int(lm.y * h)
            confidence = lm.visibility

            # Update bounding box coordinates
            x_min = min(x_min, cx)
            y_min = min(y_min, cy)
            x_max = max(x_max, cx)
            y_max = max(y_max, cy)

            # Draw circles at landmarks
            cv2.circle(img, (cx, cy), 10, (255, 0, 0), cv2.FILLED)

        # Draw bounding box
        cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

    # Calculate and display FPS
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime

    cv2.putText(img, f'FPS: {int(fps)}', (70, 50), cv2.FONT_HERSHEY_PLAIN, 3, (255, 255, 255), 3)

    # Resize and write frame
    img_resized = cv2.resize(img, (width, height))
    out.write(img_resized)

cap.release()
out.release()

# Download the processed video
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# CSV generation
import cv2
import mediapipe as mp
import time
import csv
from google.colab import files

# Initialize MediaPipe Pose
mpDraw = mp.solutions.drawing_utils
mpPose = mp.solutions.pose
pose = mpPose.Pose()

# Function to process video and generate CSV
def process_video_and_generate_csv(input_path, csv_path):
    print(f"Opening video file: {input_path}")
    cap = cv2.VideoCapture(input_path)

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    duration = total_frames / fps

    print(f"Video properties: {width}x{height} at {fps} fps")
    print(f"Total frames to process: {total_frames}, Duration: {duration:.2f}s")

    with open(csv_path, mode='w', newline='') as csv_file:
        writer = csv.writer(csv_file)

        # CSV Header
        header = [
            'frame_index', 'video_time(s)', 'person_id',
            'bounding_box_x1', 'bounding_box_y1', 'bounding_box_x2', 'bounding_box_y2'
        ]
        for i in range(33):  # MediaPipe provides 33 keypoints
            header.extend([f'keypoint_{i}_x', f'keypoint_{i}_y', f'keypoint_{i}_z', f'keypoint_{i}_visibility'])
        writer.writerow(header)

        frame_count = 0

        while cap.isOpened():
            ret, img = cap.read()
            if not ret:
                break

            frame_count += 1
            video_time = frame_count / fps

            # Convert the frame to RGB and process with MediaPipe Pose
            imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = pose.process(imgRGB)

            if results.pose_landmarks:
                h, w, _ = img.shape
                x_min, y_min = w, h
                x_max, y_max = 0, 0

                keypoints = []
                for id, lm in enumerate(results.pose_landmarks.landmark):
                    cx, cy, cz = lm.x * w, lm.y * h, lm.z
                    confidence = lm.visibility

                    # Update bounding box coordinates
                    x_min = min(x_min, int(cx))
                    y_min = min(y_min, int(cy))
                    x_max = max(x_max, int(cx))
                    y_max = max(y_max, int(cy))

                    # Append keypoint data
                    keypoints.extend([cx, cy, cz, confidence])

                # Bounding box
                bounding_box = [x_min, y_min, x_max, y_max]

                # Write row to CSV
                row = [frame_count, video_time, 0] + bounding_box + keypoints
                writer.writerow(row)

            if frame_count % 10 == 0:
                progress = (frame_count / total_frames) * 100
                print(f'Processing: {progress:.1f}% complete ({frame_count}/{total_frames} frames)', end='\r')

    cap.release()
    print("\nCSV generation complete!")
    print(f"Saved CSV to: {csv_path}")

# Upload video file
uploaded = files.upload()
input_video_path = next(iter(uploaded.keys()))
output_csv_path = 'MediaPipe_Keypoints_Output.csv'

# Process the video and generate the CSV
process_video_and_generate_csv(input_video_path, output_csv_path)

# Download the CSV file
files.download(output_csv_path)


Saving 140.mp4 to 140 (1).mp4
Opening video file: 140 (1).mp4
Video properties: 900x900 at 50.0 fps
Total frames to process: 490, Duration: 9.80s
Processing: 100.0% complete (490/490 frames)
CSV generation complete!
Saved CSV to: MediaPipe_Keypoints_Output.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
